## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append('../')
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Kemijarvi,66.7131,27.4306,30.11,97,100,10.63,FI,snow,2021-04-23 02:05:18
1,1,Ushuaia,-54.8000,-68.3000,41.00,93,75,11.50,AR,broken clouds,2021-04-23 02:04:34
2,2,Cernay,47.8097,7.1770,43.00,65,0,5.75,FR,clear sky,2021-04-23 02:05:18
3,3,Poum,-20.2333,164.0167,79.00,77,100,5.91,NC,overcast clouds,2021-04-23 02:05:18
4,4,New Norfolk,-42.7826,147.0587,57.00,67,100,5.01,AU,overcast clouds,2021-04-23 02:05:19


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 67
What is the maximum temperature you would like for your trip? 78


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
11,11,Ambilobe,-13.2000,49.0500,72.45,88,99,3.53,MG,overcast clouds,2021-04-23 02:05:21
33,33,Praia,14.9215,-23.5087,68.00,77,20,16.11,CV,few clouds,2021-04-23 02:05:28
34,34,Bongandanga,1.5000,21.0500,69.33,96,89,1.66,CD,light rain,2021-04-23 02:05:28
41,41,Along,28.1667,94.7667,70.84,85,71,1.10,IN,broken clouds,2021-04-23 02:05:30
43,43,Bambous Virieux,-20.3428,57.7575,73.99,94,40,8.05,MU,light rain,2021-04-23 02:05:31
...,...,...,...,...,...,...,...,...,...,...,...
671,671,Ferkessedougou,9.6000,-5.2000,74.82,79,100,6.96,CI,overcast clouds,2021-04-23 02:08:45
682,682,Kaihua,23.3687,104.2800,73.11,66,40,13.27,CN,scattered clouds,2021-04-23 02:07:21
687,687,Cap Malheureux,-19.9842,57.6142,73.99,84,100,4.00,MU,moderate rain,2021-04-23 02:08:49
692,692,Mangan,27.5167,88.5333,72.61,30,0,1.03,IN,clear sky,2021-04-23 02:08:50


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             127
City                   127
Lat                    127
Lng                    127
Max Temp               127
Humidity               127
Cloudiness             127
Wind Speed             127
Country                127
Current Description    127
Date                   127
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Ambilobe,MG,72.45,overcast clouds,-13.2000,49.0500,
33,Praia,CV,68.00,few clouds,14.9215,-23.5087,
34,Bongandanga,CD,69.33,light rain,1.5000,21.0500,
41,Along,IN,70.84,broken clouds,28.1667,94.7667,
43,Bambous Virieux,MU,73.99,light rain,-20.3428,57.7575,
...,...,...,...,...,...,...,...
671,Ferkessedougou,CI,74.82,overcast clouds,9.6000,-5.2000,
682,Kaihua,CN,73.11,scattered clouds,23.3687,104.2800,
687,Cap Malheureux,MU,73.99,moderate rain,-19.9842,57.6142,
692,Mangan,IN,72.61,clear sky,27.5167,88.5333,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df           

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Ambilobe,MG,72.45,overcast clouds,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
33,Praia,CV,68.00,few clouds,14.9215,-23.5087,LT Aparthotel
34,Bongandanga,CD,69.33,light rain,1.5000,21.0500,
41,Along,IN,70.84,broken clouds,28.1667,94.7667,Hotel West
43,Bambous Virieux,MU,73.99,light rain,-20.3428,57.7575,Casa Tia Villa
...,...,...,...,...,...,...,...
671,Ferkessedougou,CI,74.82,overcast clouds,9.6000,-5.2000,Hotel Chateau
682,Kaihua,CN,73.11,scattered clouds,23.3687,104.2800,Zhongyu Hotel
687,Cap Malheureux,MU,73.99,moderate rain,-19.9842,57.6142,Pereybere Hotel & Spa
692,Mangan,IN,72.61,clear sky,27.5167,88.5333,Hotel Mout View


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Ambilobe,MG,72.45,overcast clouds,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
33,Praia,CV,68.00,few clouds,14.9215,-23.5087,LT Aparthotel
34,Bongandanga,CD,69.33,light rain,1.5000,21.0500,
41,Along,IN,70.84,broken clouds,28.1667,94.7667,Hotel West
43,Bambous Virieux,MU,73.99,light rain,-20.3428,57.7575,Casa Tia Villa
...,...,...,...,...,...,...,...
671,Ferkessedougou,CI,74.82,overcast clouds,9.6000,-5.2000,Hotel Chateau
682,Kaihua,CN,73.11,scattered clouds,23.3687,104.2800,Zhongyu Hotel
687,Cap Malheureux,MU,73.99,moderate rain,-19.9842,57.6142,Pereybere Hotel & Spa
692,Mangan,IN,72.61,clear sky,27.5167,88.5333,Hotel Mout View


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))